# Modèles de Raisonnement : o4-mini et GPT-5-thinking

Les modèles de raisonnement représentent une évolution majeure des LLMs. Contrairement aux modèles de chat classiques, ils prennent le temps de "réfléchir" avant de répondre, ce qui améliore significativement leurs performances sur les tâches complexes.

**Objectifs :**
- Comprendre les différences architecturales (thinking time)
- Maîtriser `reasoning_effort` (low, medium, high)
- Choisir le bon modèle selon la tâche
- Analyser les performances et coûts

**Prérequis :** Notebook 2 (Prompt Engineering)

**Durée estimée :** 60 minutes

In [ ]:
%pip install openai python-dotenv --quiet

import os
import time
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv('../.env')
client = OpenAI()

# Modèle par défaut depuis .env
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

print(f"Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")
print(f"Mode: {'BATCH' if BATCH_MODE else 'INTERACTIF'}")

## 1. Chat Models vs Reasoning Models

### Différences fondamentales

| Aspect | Chat Models | Reasoning Models |
|--------|-------------|------------------|
| **Réponse** | Immédiate | Temps de réflexion |
| **Optimisation** | Dialogue fluide | Problèmes complexes |
| **Exemples** | gpt-4o, gpt-4o-mini | o4-mini, gpt-5-thinking |
| **Vitesse** | Rapide (1-3s) | Variable (5-30s+) |
| **Précision** | Bonne | Excellente sur tâches complexes |

### Quand utiliser quoi ?

**Chat Models :**
- Conversations naturelles
- Questions factuelles simples
- Génération de contenu créatif
- Traduction, résumé

**Reasoning Models :**
- Mathématiques et logique
- Programmation complexe
- Analyse multi-étapes
- Problèmes d'optimisation
- Déduction et inférence

In [2]:
probleme_math = """
Alice a 5 pommes. Elle en donne la moitié à Bob.
Bob mange 1 pomme puis rend le reste à Alice.
Alice achète ensuite 3 pommes de plus.
Combien de pommes Alice a-t-elle maintenant?
Donne uniquement le nombre final.
"""

# Test avec gpt-4o-mini (chat model)
start = time.time()
response_chat = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": probleme_math}],
    max_tokens=100
)
time_chat = time.time() - start

# Test avec o4-mini (reasoning model)
start = time.time()
response_reasoning = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": "Formatting re-enabled"},
        {"role": "user", "content": probleme_math}
    ],
    reasoning_effort="medium"
)
time_reasoning = time.time() - start

print("=== Comparaison Chat vs Reasoning ===")
print(f"\ngpt-4o-mini ({time_chat:.2f}s):")
print(f"  Réponse: {response_chat.choices[0].message.content.strip()}")
print(f"\no4-mini ({time_reasoning:.2f}s):")
print(f"  Réponse: {response_reasoning.choices[0].message.content.strip()}")
print(f"\n[Réponse correcte: 5.5 pommes (Alice donne 2.5, récupère 1.5, achète 3 → 5 + 1.5 + 3 - 2.5 = 7)]")
print(f"\nNote: Le chat model peut faire une erreur de calcul, le reasoning model analyse pas à pas.")

=== Comparaison Chat vs Reasoning ===

gpt-4o-mini (0.78s):
  Réponse: Alice a maintenant 7 pommes.

o4-mini (4.67s):
  Réponse: 7

[Réponse correcte: 5.5 pommes (Alice donne 2.5, récupère 1.5, achète 3 → 5 + 1.5 + 3 - 2.5 = 7)]

Note: Le chat model peut faire une erreur de calcul, le reasoning model analyse pas à pas.


## 2. Paramètre `reasoning_effort`

Les modèles de raisonnement exposent un paramètre crucial : **`reasoning_effort`**.

### Niveaux disponibles

| Niveau | Temps de réflexion | Qualité | Usage typique |
|--------|-------------------|---------|---------------|
| **low** | Minimal (5-10s) | Bonne | Problèmes simples nécessitant un peu de réflexion |
| **medium** | Équilibré (10-20s) | Très bonne | Cas d'usage général |
| **high** | Approfondi (20-60s+) | Excellente | Problèmes très complexes |

### Règles empiriques

- **low** : Questions de logique simple, calculs directs
- **medium** : Programmation standard, analyses multi-étapes
- **high** : Optimisation mathématique, preuve formelle, debugging complexe

**Note importante :** Plus de réflexion = plus de coût en tokens. Choisissez judicieusement !

In [3]:
probleme_logique = """
Dans une course, tu dépasses le 2ème. 
Quelle est ta position finale?
Explique ton raisonnement en une phrase.
"""

print("=== Impact du reasoning_effort ===\n")

for effort in ["low", "medium", "high"]:
    start = time.time()
    response = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "developer", "content": "Formatting re-enabled"},
            {"role": "user", "content": probleme_logique}
        ],
        reasoning_effort=effort
    )
    duration = time.time() - start
    
    print(f"reasoning_effort='{effort}' ({duration:.2f}s):")
    print(f"  {response.choices[0].message.content.strip()}")
    print()

print("[Réponse correcte: 2ème position - tu prends la place de celui que tu dépasses]")
print("\nObservation: Les 3 niveaux devraient donner la bonne réponse, mais 'high' peut fournir")
print("une explication plus détaillée. La différence est surtout visible sur des problèmes complexes.")

=== Impact du reasoning_effort ===

reasoning_effort='low' (2.76s):
  Tu te retrouves à la 2ᵉ place, car en dépassant le coureur classé 2ᵉ tu prends sa position.

reasoning_effort='medium' (4.43s):
  Ta position finale est 2ᵉ : en dépassant celui qui était deuxième, tu prends sa place.

reasoning_effort='high' (5.95s):
  Tu termines 2e : en dépassant le 2e coureur, tu prends sa place.

[Réponse correcte: 2ème position - tu prends la place de celui que tu dépasses]

Observation: Les 3 niveaux devraient donner la bonne réponse, mais 'high' peut fournir
une explication plus détaillée. La différence est surtout visible sur des problèmes complexes.


## 3. Messages `developer` et contrôle du formatage

### Rôle `developer`

Les modèles de raisonnement (o4-mini, gpt-5-thinking) utilisent un rôle spécial : **`developer`**.

- Remplace le rôle `system` des chat models
- Définit des instructions méta (comportement, formatage)
- Moins strict que `system`, plus flexible

### "Formatting re-enabled"

Par défaut, les modèles raisonnants retournent du texte brut. Pour activer le **markdown** (formules LaTeX, code, listes), ajoutez :

```python
{"role": "developer", "content": "Formatting re-enabled"}
```

Ceci permet :
- Formules mathématiques : `$E = mc^2$`
- Blocs de code avec syntax highlighting
- Tableaux, listes, titres markdown

**Sans ce message**, vous obtenez du texte brut (pas de formatage riche).

In [4]:
# Sans "Formatting re-enabled"
response_no_format = client.chat.completions.create(
    model="o4-mini",
    messages=[{"role": "user", "content": "Explique le théorème de Pythagore avec des formules."}],
    reasoning_effort="low"
)

# Avec "Formatting re-enabled"
response_with_format = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": "Formatting re-enabled"},
        {"role": "user", "content": "Explique le théorème de Pythagore avec des formules."}
    ],
    reasoning_effort="low"
)

print("=== Sans 'Formatting re-enabled' ===")
print(response_no_format.choices[0].message.content[:400])
print("\n...\n")

print("\n=== Avec 'Formatting re-enabled' ===")
print(response_with_format.choices[0].message.content[:400])
print("\n...\n")

print("\nDifférence: Avec formatage, vous obtenez du markdown propre (formules LaTeX, code blocks, etc.)")

=== Sans 'Formatting re-enabled' ===
Énoncé du théorème  
Dans un triangle rectangle, le carré de la longueur de l’hypoténuse (le côté opposé à l’angle droit) est égal à la somme des carrés des longueurs des deux autres côtés.

Soit un triangle ABC rectangle en C, avec :  
- AB = c (hypoténuse)  
- BC = a  
- AC = b  

Alors le théorème de Pythagore s’écrit :  
(1) c² = a² + b²  

Démonstrations succinctes  

1. Preuve par découpage 

...


=== Avec 'Formatting re-enabled' ===
Le théorème de Pythagore, valable dans un triangle rectangle, énonce que :

Soit un triangle ABC rectangle en A (l’angle en A vaut 90°). On note :
- BC = c (l’hypoténuse, côté opposé à l’angle droit)  
- AB = b  
- AC = a  

Alors la relation entre les longueurs des côtés est :

1) Formule principale  
   a² + b² = c²  

2) Interprétation géométrique  
   – L’aire du carré de côté a (donc a²) plus

...


Différence: Avec formatage, vous obtenez du markdown propre (formules LaTeX, code blocks, etc.)


## 4. Cas d'usage : Génération de code complexe

Les modèles de raisonnement excellent dans la génération de code nécessitant :
- Analyse algorithmique
- Optimisation de complexité
- Tests edge cases
- Debugging multi-étapes

### Exemple : Algorithme du plus long palindrome

Problème classique d'algorithmique :
- Approche naïve : O(n³)
- Approche optimisée : O(n²) avec expansion autour du centre
- Approche avancée : O(n) avec Manacher's algorithm

Un modèle raisonnant peut analyser les différentes approches et choisir la meilleure.

In [5]:
probleme_code = """
Écris une fonction Python qui trouve le plus long palindrome dans une chaîne.
La fonction doit être optimisée (O(n²) maximum).
Inclus des tests avec plusieurs cas (chaîne vide, un seul caractère, palindrome complet, pas de palindrome).
"""

print("=== Génération de code avec o4-mini (reasoning_effort='high') ===\n")

start = time.time()
response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": "Formatting re-enabled"},
        {"role": "user", "content": probleme_code}
    ],
    reasoning_effort="high"
)
duration = time.time() - start

print(f"Temps de génération: {duration:.2f}s\n")
print(response.choices[0].message.content)

print("\n" + "="*60)
print("Note: Le modèle devrait fournir :")
print("  1. Une implémentation avec expansion autour du centre (O(n²))")
print("  2. Des tests complets couvrant les edge cases")
print("  3. Des commentaires expliquant la logique")

=== Génération de code avec o4-mini (reasoning_effort='high') ===

Temps de génération: 17.29s

Voici une solution en O(n²) qui, pour chaque position, « étend » un palindrome de longueur impaire et paire.  

```python
def longest_palindrome(s: str) -> str:
    """
    Retourne le plus long sous-palindrome de s.
    Complexité time O(n^2), space O(1) (hors sortie).
    """
    n = len(s)
    if n < 2:
        return s  # "" ou 1 caractère est déjà un palindrome

    start, max_len = 0, 1

    def expand_around_center(left: int, right: int):
        nonlocal start, max_len
        # étend tant que les bornes sont valides et que les caractères correspondent
        while left >= 0 and right < n and s[left] == s[right]:
            curr_len = right - left + 1
            if curr_len > max_len:
                start, max_len = left, curr_len
            left -= 1
            right += 1

    for i in range(n):
        # cas palindrome de longueur impaire (centre sur i)
        expand_around_

## 5. Modèles de Raisonnement Avancés

### Panorama des modèles raisonnants disponibles

| Modèle | Disponibilité | Caractéristiques |
|--------|---------------|------------------|
| **o1-mini** | Disponible | Rapide, économique, bonne précision |
| **o1-preview** | Disponible | Plus puissant, plus lent |
| **o3-mini** | Disponible | Dernière génération, équilibré |
| **o4-mini** | Disponible | Version optimisée 2026 |
| **gpt-5-thinking** | Accès limité | Très avancé, pas encore public |

### Recommandations pratiques

- **o4-mini** : Premier choix pour la plupart des tâches de raisonnement
- **o3-mini** : Alternative si o4-mini n'est pas disponible
- **gpt-5-thinking** : Réservé aux cas très complexes (accès restreint)

**Note :** La disponibilité des modèles dépend de votre niveau d'accès API. Les exemples ci-dessous utilisent les modèles accessibles publiquement.

In [ ]:
probleme_complexe = """
Résous ce problème d'optimisation:

Un fermier veut construire un enclos rectangulaire contre un mur existant.
Il a 100 mètres de clôture disponible (le mur fait office d'un côté).
Quelle doit être la dimension de l'enclos pour maximiser la surface?

Détaille toutes les étapes mathématiques :
1. Définition des variables
2. Équation de contrainte
3. Fonction à optimiser
4. Calcul de la dérivée
5. Résolution et vérification
"""

print("=== Modèle raisonnant sur problème d'optimisation ===\n")

# Essayer d'abord gpt-5-thinking, fallback sur o4-mini
models_to_try = ["gpt-5-thinking", "o4-mini", "o3-mini", "o1-preview"]

for model in models_to_try:
    try:
        print(f"Tentative avec {model}...")
        start = time.time()
        
        # Configuration différente selon le modèle
        if model.startswith("gpt-5"):
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "developer", "content": "Formatting re-enabled"},
                    {"role": "user", "content": probleme_complexe}
                ]
            )
        else:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "developer", "content": "Formatting re-enabled"},
                    {"role": "user", "content": probleme_complexe}
                ],
                reasoning_effort="high"
            )
        
        duration = time.time() - start
        print(f"Succès avec {model} ! (Temps: {duration:.2f}s)\n")
        print(response.choices[0].message.content)
        break  # Sortir de la boucle si succès
        
    except Exception as e:
        print(f"  {model} non disponible: {type(e).__name__}")
        if model == models_to_try[-1]:
            print("\nAucun modèle raisonnant disponible. Utilisation de gpt-4o en fallback.")
            response = client.chat.completions.create(
                model=DEFAULT_MODEL,
                messages=[{"role": "user", "content": probleme_complexe}]
            )
            print(response.choices[0].message.content)

print("\n" + "="*60)
print("Réponse correcte: Longueur 50m (parallèle au mur), Largeur 25m, Surface 1250m²")
print("Équation: S(x) = x(100-2x), dérivée S'(x) = 100 - 4x, max en x=25")

## 6. Comparaison coût/performance

### Tarification (approximative, vérifiez les tarifs actuels)

| Modèle | Input ($/1M tokens) | Output ($/1M tokens) | Vitesse | Précision |
|--------|---------------------|----------------------|---------|----------|
| **gpt-4o-mini** | 0.15 | 0.60 | ⚡⚡⚡ | ⭐⭐⭐ |
| **gpt-4o** | 2.50 | 10.00 | ⚡⚡ | ⭐⭐⭐⭐ |
| **o4-mini** | 1.50 | 6.00 | ⚡⚡ | ⭐⭐⭐⭐⭐ (problèmes complexes) |
| **gpt-5-thinking** | 5.00 | 20.00 | ⚡ | ⭐⭐⭐⭐⭐ (très complexe) |

### Guide de choix

```python
def choisir_modele(tache):
    if tache.type == "conversation" or tache.complexite == "faible":
        return "gpt-4o-mini"  # Rapide, économique
    
    elif tache.type == "generation_creative" or tache.complexite == "moyenne":
        return "gpt-4o"  # Équilibre qualité/vitesse
    
    elif tache.type in ["math", "code", "logique"] and tache.complexite == "élevée":
        return "o4-mini"  # Raisonnement économique
    
    elif tache.complexite == "très élevée" or tache.precision_requise == "maximale":
        return "gpt-5-thinking"  # Pour les cas difficiles
    
    else:
        return "gpt-4o"  # Par défaut
```

### Tokens de raisonnement

Les modèles raisonnants génèrent des **tokens de réflexion** (non visibles) avant la réponse finale.
- `reasoning_effort="low"` : ~500-1000 tokens de réflexion
- `reasoning_effort="medium"` : ~1000-3000 tokens
- `reasoning_effort="high"` : ~3000-10000+ tokens

**Ces tokens sont facturés** ! Optimisez selon vos besoins.

## 7. Benchmark : Chat vs Reasoning

Testons les deux approches sur plusieurs problèmes types.

In [7]:
import statistics

problemes = [
    ("Combien font 17 * 23?", "391"),
    ("Si 3 chats attrapent 3 souris en 3 minutes, combien de chats faut-il pour attraper 100 souris en 100 minutes?", "3 chats"),
    ("Un train part de Paris à 8h et roule à 120 km/h. Un autre part de Lyon (450km) à 9h à 150 km/h vers Paris. À quelle heure se croisent-ils?", "environ 10h30")
]

print("=== Benchmark Chat vs Reasoning ===\n")

temps_chat = []
temps_reasoning = []

for i, (prob, correct) in enumerate(problemes):
    print(f"Problème {i+1}: {prob[:60]}...")
    
    # Chat model
    start = time.time()
    resp_chat = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prob + " Réponds uniquement avec le résultat."}],
        max_tokens=50
    )
    t_chat = time.time() - start
    temps_chat.append(t_chat)
    
    # Reasoning model
    start = time.time()
    resp_reason = client.chat.completions.create(
        model="o4-mini",
        messages=[{"role": "user", "content": prob + " Réponds uniquement avec le résultat."}],
        reasoning_effort="medium"
    )
    t_reason = time.time() - start
    temps_reasoning.append(t_reason)
    
    print(f"  gpt-4o-mini ({t_chat:.2f}s): {resp_chat.choices[0].message.content.strip()}")
    print(f"  o4-mini ({t_reason:.2f}s): {resp_reason.choices[0].message.content.strip()}")
    print(f"  [Correct: {correct}]\n")

print("\n=== Statistiques ===")
print(f"Temps moyen gpt-4o-mini: {statistics.mean(temps_chat):.2f}s")
print(f"Temps moyen o4-mini: {statistics.mean(temps_reasoning):.2f}s")
print(f"\nObservation: Les modèles raisonnants sont plus lents mais généralement plus précis")
print(f"sur les problèmes nécessitant plusieurs étapes de calcul.")

=== Benchmark Chat vs Reasoning ===

Problème 1: Combien font 17 * 23?...
  gpt-4o-mini (0.58s): 391
  o4-mini (1.61s): 391
  [Correct: 391]

Problème 2: Si 3 chats attrapent 3 souris en 3 minutes, combien de chats...
  gpt-4o-mini (1.20s): Il faut 3 chats.
  o4-mini (3.02s): 3
  [Correct: 3 chats]

Problème 3: Un train part de Paris à 8h et roule à 120 km/h. Un autre pa...
  gpt-4o-mini (1.73s): Ils se croisent à 10h30.
  o4-mini (8.60s): 10 h 13 min 20 s
  [Correct: environ 10h30]


=== Statistiques ===
Temps moyen gpt-4o-mini: 1.17s
Temps moyen o4-mini: 4.41s

Observation: Les modèles raisonnants sont plus lents mais généralement plus précis
sur les problèmes nécessitant plusieurs étapes de calcul.


## 8. Exemple pratique : Débogage de code

Les modèles de raisonnement excellent dans le debugging complexe.

In [8]:
code_buggue = '''
def fibonacci(n):
    """Calcule le n-ième nombre de Fibonacci."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Test
for i in range(40):
    print(f"F({i}) = {fibonacci(i)}")
'''

prompt_debug = f"""
Analyse ce code Python et identifie le(s) problème(s) :

```python
{code_buggue}
```

Le code fonctionne mais devient TRÈS lent pour n >= 35.

1. Identifie le problème de performance
2. Explique pourquoi (complexité algorithmique)
3. Propose une solution optimisée
4. Fournis le code corrigé
"""

print("=== Débogage avec o4-mini (reasoning_effort='high') ===\n")

response = client.chat.completions.create(
    model="o4-mini",
    messages=[
        {"role": "developer", "content": "Formatting re-enabled"},
        {"role": "user", "content": prompt_debug}
    ],
    reasoning_effort="high"
)

print(response.choices[0].message.content)

print("\n" + "="*60)
print("Le modèle devrait identifier:")
print("  - Complexité O(2^n) due aux appels récursifs redondants")
print("  - Solution: mémoïsation ou approche itérative (O(n))")

=== Débogage avec o4-mini (reasoning_effort='high') ===

Voici une analyse point par point et une version optimisée :

1. Problème de performance  
   - La fonction fibonacci fait deux appels récursifs à chaque niveau.  
   - Elle recalcule sans cesse les mêmes valeurs de Fibonacci.

2. Pourquoi c’est lent (complexité)  
   - L’arbre d’appels récursifs double quasiment à chaque niveau.  
   - En notation Big-O, la complexité est O(2ⁿ).  
   - Pour n = 35–40, le nombre d’appels explose (plusieurs dizaines de millions).

3. Solution optimisée  
   - utiliser un cache (mémoisation) pour ne calculer chaque F(k) qu’une seule fois, ou  
   - passer à une version itérative en O(n) et O(1) d’espace supplémentaire.

4. Code corrigé  

Option A – avec mémoisation automatique :

```python
from functools import lru_cache

@lru_cache(maxsize=None)
def fibonacci(n):
    """Calcule le n-ième nombre de Fibonacci en mémoisant."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    els

## 9. Limites et considérations

### Limites des modèles de raisonnement

1. **Temps de réponse** : Peuvent être lents (30-60s+) pour `reasoning_effort="high"`
2. **Coût** : Tokens de réflexion facturés même s'ils ne sont pas visibles
3. **Pas toujours nécessaires** : Sur-utiliser peut gaspiller temps et argent
4. **Pas magiques** : Ne garantissent pas la correction (vérifiez toujours les résultats)

### Bonnes pratiques

1. **Commencez simple** : Testez d'abord avec un chat model
2. **Montez progressivement** : gpt-4o-mini → gpt-4o → o4-mini → gpt-5-thinking
3. **Ajustez reasoning_effort** : Commencez par "low", augmentez si nécessaire
4. **Validez les résultats** : Surtout pour le code et les calculs critiques
5. **Mesurez le ROI** : Le temps/coût supplémentaire est-il justifié ?

### Cas où les chat models suffisent

- Conversations naturelles
- Résumés et synthèses
- Traduction simple
- Génération de contenu créatif
- Questions factuelles directes
- Code simple (CRUD, scripts basiques)

## 10. Conclusion et ressources

### Points clés à retenir

1. **Modèles de raisonnement** = temps de réflexion + meilleure précision sur tâches complexes
2. **o4-mini** : Excellent rapport qualité/prix pour raisonnement standard
3. **gpt-5-thinking** : Pour les problèmes très complexes nécessitant raisonnement profond
4. **reasoning_effort** : Contrôle le compromis temps/qualité (low/medium/high)
5. **Messages developer** : Remplacent `system`, `"Formatting re-enabled"` active le markdown
6. **Choisir judicieusement** : Ne pas sur-utiliser les modèles raisonnants (coût)

### Tableau récapitulatif

| Tâche | Modèle recommandé | reasoning_effort |
|-------|-------------------|------------------|
| Conversation | gpt-4o-mini | N/A |
| Code simple | gpt-4o | N/A |
| Algorithme complexe | o4-mini | medium |
| Optimisation math | o4-mini | high |
| Preuve formelle | gpt-5-thinking | N/A (auto) |
| Debugging approfondi | o4-mini ou gpt-5-thinking | high |

### Exercices suggérés

1. **Comparaison** : Testez un problème de logique avec gpt-4o-mini vs o4-mini
2. **Optimisation** : Demandez à o4-mini d'optimiser un algorithme de tri
3. **Reasoning effort** : Comparez low/medium/high sur un problème d'optimisation
4. **Code generation** : Demandez l'implémentation d'un algorithme complexe (Dijkstra, A*)
5. **Multi-étapes** : Problème nécessitant analyse → planification → exécution

### Ressources

- [Documentation OpenAI - Reasoning Models](https://platform.openai.com/docs/guides/reasoning)
- [Guide des tarifs](https://openai.com/pricing)
- [Best practices pour o4-mini](https://platform.openai.com/docs/guides/reasoning/best-practices)
- Notebook suivant : **9_Vision_Models.ipynb** (GPT-4 Vision, analyse d'images)

### Prochaines étapes

Dans le prochain notebook, nous explorerons les **modèles de vision** :
- GPT-4 Vision (gpt-4o avec images)
- Analyse de screenshots, diagrammes, graphiques
- Extraction de texte (OCR)
- Génération de descriptions d'images